<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Attribution function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Attribution refers to the process of assigning credit or responsibility to a specific event or entity that contributes to an outcome of interest.Specifically, the Attribution function is used for web page analysis, which refers to the process of assigning value or credit to different pages on a website for specific actions taken by visitors, such as making a purchase or filling out a form. The goal of attribution is to identify the most effective pages or content on a website that contribute to achieving business goals. By assigning weights or credit to different pages, organizations can optimize their website by improving or eliminating underperforming pages and investing more resources into the most effective ones. Attribution can be done using various methods, including rule-based attribution and data-driven attribution.<br>In this notebook we will see how we can use the Attribution function available in Vantage.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_Attribution_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_Financial_cloud');"        # Takes 30 seconds
#%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_Financial_local');" 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
tdf = DataFrame(in_schema('DEMO_Financial', 'Customer_journey'))
print("Shape of the data: ", tdf.shape)
tdf

In [ ]:
tdf.tdtypes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Attribution function takes data and parameters from multiple tables and outputs attributions. Please refer to Teradata Vantage™ - Analytics Database Analytic Functions documentation for more on Attribution function or use help(Attribution)</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Attribution Input :
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Input tables (maximum of five) (Contain data for computing attributions).</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>ConversionEventTable (Contains conversion events).</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>FirstModelTable (Defines type and distributions of model - we'll create one table per model)</li></ol>
</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Attribution Syntax Elements:
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>EventColumn specifies the name of the input column that contains the events.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>TimeColumn specifies the name of the input column that contains the timestamps of the  events.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>WindowSize specifies how to determine the maximum window size for the attribution calculation</li></ol>
    </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will create the model and conversion tables that allow us to send large numbers of parameters programmatically to the Attribution analytic function.</p>

<p style = 'font-size:14px;font-family:Arial;color:#00233C'><i>Note: We are using SQLs to create the needed tables and insert the required values in these tables</i></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Detailed help can be found by passing function name to built-in help function. </p>

In [ ]:
help(Attribution)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For the input to our Attribution function, let's create Conversion event table i.e the events we want to track and the Model table for the type of attribution model we want to use.</p>

In [ ]:
query1 = '''
CREATE MULTISET TABLE conversion_events (
    conversion_event VARCHAR(55)
)
NO PRIMARY INDEX;
'''
query2 = '''
INSERT INTO conversion_events VALUES('ACCOUNT_BOOKED_ONLINE');
INSERT INTO conversion_events VALUES('ACCOUNT_BOOKED_OFFLINE');
'''
execute_sql(query1)
execute_sql(query2)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In our Conversion event table we have added 'ACCOUNT_BOOKED_ONLINE' and 'ACCOUNT_BOOKED_OFFLINE' events as we want to assign attribution score based on these events. Next is to assign attribution model</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Following methods available in Attribution:
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Uniform: Conversion event is attributed uniformly to preceding attributable events.</li>
    <li>First Click: Conversion event is attributed entirely to first attributable event.</li>
    <li>Last Click: Conversion event is attributed entirely to most recent attributable event</li> 
    <li>Exponential:  Conversion event is attributed exponentially to preceding attributable events (the more recent the event, the higher the attribution).</li>
 </ul>
</p>

In [ ]:
query1 = '''
CREATE MULTISET TABLE attribution_model (
    id    INTEGER,
    model VARCHAR(100)
)
NO PRIMARY INDEX;
'''
query2 = '''
INSERT INTO attribution_model VALUES(0,'SEGMENT_ROWS');
INSERT INTO attribution_model VALUES(1,'3:0.5:UNIFORM:NA');
INSERT INTO attribution_model VALUES(2,'4:0.3:LAST_CLICK:NA');
INSERT INTO attribution_model VALUES(3,'3:0.2:FIRST_CLICK:NA');
'''
execute_sql(query1)
execute_sql(query2)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the above Attribution Model the attribution score is assigned as followed. Attribution for a conversion event is divided among attributable events in 10 rows immediately preceding conversion event. <br>If conversion event is in row 11, first model specification applies to rows 10, 9, and 8; second applies to rows 7, 6, 5, and 4; and third applies to rows 3, 2, and 1.<br>
Half attribution (5/10) is uniformly divided among rows 10, 9, and 8; 3/10 to last click in rows 7, 6, 5, and 4 (that is, in row 7), and 2/10 to first click in rows 3, 2, and 1.

In [ ]:
ConvEvent_df = DataFrame("conversion_events")
FirstModel_df = DataFrame("attribution_model") 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Attribution function requires the datatype of the Event column to be Latin, hence we convert our data from Unicode to Latin.</p>

In [ ]:
from teradataml import ConvertTo
converted_data = ConvertTo(data = tdf,
                           target_columns = ['interaction_type'],
                           target_datatype = ["VARCHAR(charlen=100,charset=LATIN,casespecific=NO)"])
convert_tdf=converted_data.result
convert_tdf.to_sql('convert_tdf')

In [ ]:
attribution_out = Attribution(data=DataFrame('convert_tdf'),
                              data_partition_column="customer_identifier",
                              data_order_column="interaction_timestamp",
                              event_column="interaction_type",
                              conversion_data=ConvEvent_df,
                              timestamp_column = "interaction_timestamp",
                              window_size = "rows:10",
                              model1_type=FirstModel_df)

Attrdf = attribution_out.result
Attrdf.sort('attribution',ascending=False)

In [ ]:
print(attribution_out.show_query())

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Cleanup</b>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
tables = ['conversion_events', 'attribution_model', 'convert_tdf']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name = table, schema_name = 'demo_user') 
    except:
        pass

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_Financial');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Dataset:</b>

`Customer_Journey`

- `customer_skey`: customer key
- `customer_identifier`: unique customer identifier
- `customer_cookie`: cookie placed on customers device
- `customer_online_id`: boolean - does the customer have an online account
- `customer_offline_id`: customer account number
- `customer_type`: is this a high value customer or just a visitor browsing the website?
- `customer_days_active`: how long has the customer been active
- `interaction_session_number`: session identifier
- `interaction_timestamp`: timestamp for this event
- `interaction_source`: channel this event is from (online / offline, in branch etc.)
- `interaction_type`: type of event
- `sales_channel`: channel a sales event was in
- `conversion_id`: sales conversion identifier
- `product_category`: what type of product the event concerned (checking, savings, cd etc..)
- `product_type`: unused
- `conversion_sales`: unused
- `conversion_cost`: unused
- `conversion_margin`: unused
- `conversion_units`: unused
- `marketing_code`: marketing identifier
- `marketing_category`: marketing channel (inbranch, website, email etc..)
- `marketing_description`: marketing campaign name
- `marketing_placement`: specific marketing outlet (Google, Bloomberg.com etc..)
- `mobile_flag`: boolean was on a mobile device
- `updt`: unused

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>Attribution function reference: <a href = 'https://docs.teradata.com/search/all?query=Attribution&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>